## Human in the loop

In the previous notebook, we created an assistant agent that can send hello-world message using LLM.

In this notebook, we will further introduce human in the loop to have conversation with that assistant agent. To do that, we are going to add another user agent that can take input from user and send it to the assistant agent. We also need to preserve the previous conversation history to make the conversation more meaningful.

In [1]:
from typing import List, Optional
from autogen_core.base import MessageContext, AgentId, AgentInstantiationContext, TopicId
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler, TypeSubscription
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile
from autogen_core.components.tool_agent import ToolAgent, tool_agent_caller_loop
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient
import random

from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool, ToolSchema
from autogen_core.components._image import Image
from typing_extensions import Annotated
from pydantic import BaseModel
from PIL import Image as PILImage
import os
import uuid;

C:\Users\xiaoyuz\AppData\Local\Temp\ipykernel_112972\1610075179.py:15: FutureWarning: OpenAIChatCompletionClient moved to autogen_ext. Please import it from autogen_ext.modelsChatCompletionClient.
  from autogen_core.components.models import OpenAIChatCompletionClient


# Load .env

In [2]:
%load_ext dotenv
%dotenv .env

# Message Contract

Here we introduce two contracts
- Message: the same one as in the previous notebook
- Conversation: the chat history

## Why we introduce Conversation contract?
The `Conversation` will save all chat history between the user and the assistant agent to provide context for assistant agent to generate more meaningful responses.

In [3]:
class Message(BaseModel):
    text: str
    source: Optional[str] = None

class Conversation(BaseModel):
    chat_history: List[Message]

## UserAgent
We first define the UserAgent class, which represents the human user in the conversation.

The UserAgent handles Conversation in the following way. When it receives a conversation, it first prompts the user for a response.
If user says `TERMINATE`, the conversation is terminated. Otherwise, the user's response is added to the conversation and publish to agent runtime.

In [4]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, assistant_topic_type: str) -> None:
        super().__init__(description=description)
        self._assistant_topic_type = assistant_topic_type

    @message_handler
    async def handle_request_to_speak(self, message: Conversation, ctx: MessageContext) -> None:
        # We don't need to respond when the last message is from the user
        if len(message.chat_history) > 0 and message.chat_history[-1].source == self.type:
            return
        user_input = input("Enter your message, type 'TERMINATE' to terminate the task: ")
        print(f"### User: \n{user_input}")
        if user_input == "TERMINATE": # end the conversation
            return
        
        message.chat_history.append(Message(text=user_input, source=self.type))
        await self.publish_message(
            message,
            DefaultTopicId(type=self._assistant_topic_type),
        )

## Assistant

The Assistant class is the agent that interacts with the user. It receives the conversation from the UserAgent, then uses LLM to generate a response. The response is then added to the conversation and published to the UserAgent.

In [5]:
class Assistant(RoutedAgent):
    def __init__(self,
                 model_client: ChatCompletionClient,
                 system_message: str = "You are a helpful AI assistant.",
                 user_topic_type: str = "user") -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._system_message = system_message
        self._user_topic_type = user_topic_type

    @message_handler
    async def handle_message(self, message: Conversation, ctx: MessageContext) -> None:
        chat_history: List[LLMMessage] = [SystemMessage(content=self._system_message)]
        last_message_source = message.chat_history[-1].source if len(message.chat_history) > 0 else self._user_topic_type
        for msg in message.chat_history:
            if msg.source == self.type:
                chat_history.append(AssistantMessage(content=msg.text, source="assistant"))
            elif msg.source == self._user_topic_type:
                chat_history.append(UserMessage(content=msg.text, source="user"))
            else:
                raise ValueError(f"Unknown message source: {msg.source}")
            
        result = await self._model_client.create(chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        message.chat_history.append(Message(text=result.content, source=self.type))
        await self.publish_message(message, topic_id=DefaultTopicId(type=last_message_source))

# Create runtime and add user, assistant agent to the runtime

## What does `add_subscription` do?
`add_subscription` tells agent runtime how to deliver messages to the agent based on the message's topic type.
In the following code
```python
await runtime.add_subscription(
    TypeSubscription("user", user_agent_type.type))
```

The runtime will deliver messages with topic type `user` to the UserAgent.

In [6]:
runtime = SingleThreadedAgentRuntime()
user_agent_type = await UserAgent.register(
    runtime,
    type="user",
    factory=lambda: UserAgent(
        description="A user agent that generates messages for the assistant agent.",
        assistant_topic_type="assistant"
    ),
)

assistant_agent_type = await Assistant.register(
    runtime,
    type="assistant",
    factory=lambda: Assistant(
        model_client=OpenAIChatCompletionClient(
            api_key=os.environ.get("OPENAI_API_KEY"),
            model="gpt-4o-mini"
        ),
        user_topic_type="user"
    ),
)

await runtime.add_subscription(
    TypeSubscription("user", user_agent_type.type))
await runtime.add_subscription(
    TypeSubscription("assistant", assistant_agent_type.type))

runtime.start()
session_id = str(uuid.uuid4())
msg = Conversation(chat_history=[])
await runtime.publish_message(
    msg,
    TopicId(type="assistant", source=session_id),
)
await runtime.stop_when_idle()



--------------------------------------------------------------------------------
Assistant:
How can I assist you today?
### User: 
My name is Geno, how are you

--------------------------------------------------------------------------------
Assistant:
Hello, Geno! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
### User: 
Tell a joke

--------------------------------------------------------------------------------
Assistant:
Sure! Here’s one for you:

Why did the scarecrow win an award? 

Because he was outstanding in his field! 

Hope that brought a smile! Want to hear another one?
### User: 
which one is larger, 9.9 or 9.11

--------------------------------------------------------------------------------
Assistant:
9.11 is larger than 9.9. The decimal 11 is greater than the decimal 9, so 9.11 > 9.9.
### User: 
summarize conversation

--------------------------------------------------------------------------------
Assista

## What we learn
- How to preserve conversation history by creating a new contract `Conversation`
- How to create a UserAgent and take input from user
- How to start a conversation between user and assistant agent